# env config

## new notebook


In [ ]:
import os
import sys
import importlib
import shutil
import json
from datetime import datetime

import pandas as pd
import numpy as np

username = 'guxia'
repo_dir = f'/projects/gds-focus/data/{username}/UCC24/'
secret_dir = f'/projects/{username}/secret'
os.chdir(os.path.join(repo_dir, 'varsel'))


for p in [secret_dir, repo_dir]:
    if p in sys.path:
        continue
    sys.path.append(p)


import credentials
importlib.reload(credentials)
from credentials import PAZ_PASSWORD



In [ ]:
!echo $PAZ_PASSWORD | kinit -l 96h guxia

In [ ]:
%reload_ext cloudmagics.auth_magics
%reload_ext cloudmagics.bigquery
%ppauth
%url -c horton
%config PPMagics.domain="ccg24-hrzana-gds-focus"
%config PPMagics.autolimit=0


In [ ]:
# unlimit query return records num
%config PPMagics.autolimit=0

# config env in notebook

```shell
cat <<EOT >> /home/$(whoami)/.bashrc

# added by guxia
# alias
alias ll='ls -l'
alias clc='clear'
EOT

source /home/$(whoami)/.bashrc 
```


# generate notebook using template

In [ ]:
model_names = ['model']

In [ ]:
import nbformat


for index, model_name in enumerate(model_names):
    nb = nbformat.read('model_perf.ipynb', as_version=4)
    inds = {}
    print(model_name )
    
    for ind, k in enumerate(nb['cells']):
        if k['cell_type'] == 'code':
            # model name
            if 'model_name = ' in k['source']:
                inds['model_name'] = ind
            # scores
            if 'ckpts' in k['source'] and 'score_list' in k['source']:
                inds['scores'] = ind
                
    print(inds)
    

    nb['cells'][inds['model_name']]['source'] = """
model_name = '{}'
""".format(model_name)

    if index == 0:
        nb['cells'][inds['scores']]['source'] = """
ckpts = os.listdir(model_ckpt_path + "/" + model_name)
score_list = [os.path.join(model_name, x) for x in ckpts if x.endswith(".ckpt")]

score_list.extend([
    'driver_ucc_sf_static_rmr_score',
    'driver_ucc21_model_score1',
    'driver_ucc_latam_cbp_model_score1',
    'driver_ucc_cbp_latam_23model',
    'driver_cam22_ems_score',
])

print('score list')
print(score_list)
"""
    else:
        nb['cells'][inds['scores']]['source'] = """
ckpts = os.listdir(model_ckpt_path + "/" + model_name)
score_list = [os.path.join(model_name, x) for x in ckpts if x.endswith(".ckpt")]

print('score list')
print(score_list)
"""
    os.makedirs('bacth_perf', exist_ok=True)
    with open(f'bacth_perf/{model_name}_perf.ipynb', 'w') as f:
        nbformat.write(nb, f)



# authenticate user
keytab file generate: go to cosmos notebook server: /projects/{username}/secret, and run `sh keytab_generator.sh {your username} {your ldap password}`
- can not authenticate user in pipeline running mode

In [ ]:
from airflownb.token import RefreshToken

cmd = f"kinit -kt /projects/guxia/secret/guxia.keytab -p guxia"
refresh = RefreshToken(cmd = cmd, interval=60 * 10)
refresh.start()